<a href="https://colab.research.google.com/github/ekomissarov/data-analysis/blob/main/20_python_many_hypothesis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://github.com/FUlyankin/matstat-AB/blob/main/week11_AB/05_python_many_hyp.ipynb

In [1]:
import numpy as np
import pandas as pd
from scipy import stats
import seaborn as sns
import matplotlib.pyplot as plt

plt.style.use('ggplot')

# Песнь льда и пламени

Говорят, Джордж Р.Р. Мартин, автор цикла "Песнь Льда и Пламени", истребляет Старков: чаще убивает персонажей, относящихся к этому дому, чем персонажей других домов. В таблице ниже приведено количество персонажей, относящихся к тому или иному дому, упомянутых за первые $4$ книги, а так же количество погибших персонажей.

| Дом | Упомянутые персонажи | Погибшие персонажи |
|-----|---------------------|------|
|House Stark	| 70 | 25 |
|House Lannister | 38 | 7 |
|House Greyjoy | 41 | 12 |
|Night's Watch |105 | 41 |

Нужно проверить гипотезу об этом на $5\%$ уровне значимости.

### Формализация задачи:

Нам нужно проверить гипотезу о том, что доля смертей среди Старков совпадает со всеми остальными долями против альтернативы, что хотя бы в одной из ситуаций смертность в доме Старков больше:

\begin{equation*}
\begin{aligned}
& H_0: p_S = p_L = p_G = p_{NW}  \\
& H_1: \text{хотя бы одна вероятность отличается}  
\end{aligned}
\end{equation*}

Протестируем все гипотезы без коррекции на множественное тестирование, а потом введём её.

In [2]:
n_st = 70
m_st = 25

n_lan = 38
m_lan = 7

n_gr = 41
m_gr = 12

n_nw = 105
m_nw = 41

alpha = 0.05

In [3]:
m_st/n_st, m_lan/n_lan, m_gr/n_gr, m_nw/n_nw

(0.35714285714285715,
 0.18421052631578946,
 0.2926829268292683,
 0.3904761904761905)

Проверяем три гипотезы!

In [4]:
from statsmodels.stats.proportion import proportions_ztest

_, p1 =  proportions_ztest((m_st, m_lan),
                           (n_st, n_lan),
                           value=0,
                           alternative='larger')


_, p2 =  proportions_ztest((m_st, m_gr),
                           (n_st, n_gr),
                           value=0,
                           alternative='larger')

_, p3 =  proportions_ztest((m_st, m_nw),
                           (n_st, n_nw),
                           value=0,
                           alternative='larger')

pval = np.array([p1, p2, p3])
pval

array([0.03008672, 0.24343199, 0.67209806])

In [5]:
pval > 0.05 # без коррекции, TRUE - не отвергается

array([False,  True,  True])

Самая первая гипотеза отвергается. Отсюда делаем вывод, что смертность значимо различается. Помним, что ошибка первого рода - ошибочный выбор альтернативы. В случае, когда мы никак не контролируем уровень значимости она оказывается завышена. Посмотрим, что будет происходить при корекции.

__а)__ Коррекция Бонферрони

In [6]:
pval > 0.05/3  # поделили уровень значимости на число гипотез :)

array([ True,  True,  True])

In [7]:
from statsmodels.sandbox.stats.multicomp import multipletests

rejected, p_corrected, _, _ = multipletests(pval,
                                            alpha = 0.05,
                                            method = 'bonferroni')
p_corrected

array([0.09026015, 0.73029598, 1.        ])

In [8]:
p_corrected > 0.05 # TRUE - не отвергается

array([ True,  True,  True])

Все три гипотезы о равенстве смертностей не отвергаются.

__б)__ Метод Холма

In [9]:
rejected, p_corrected, _, _ = multipletests(pval,
                                            alpha = 0.05,
                                            method = 'holm')
p_corrected

array([0.09026015, 0.48686399, 0.67209806])

In [10]:
p_corrected > 0.05 # TRUE - не отвергается

array([ True,  True,  True])

Все три гипотезы о равенстве смертностей не отвергаются.

__в)__ Метод Бенджамини-Хохберга

In [11]:
rejected, p_corrected, _, _ = multipletests(pval,
                                            alpha = 0.05,
                                            method = 'fdr_bh')
p_corrected

array([0.09026015, 0.36514799, 0.67209806])

In [12]:
p_corrected > 0.05 # TRUE - не отвергается

array([ True,  True,  True])

Все три гипотезы о равенстве смертностей не отвергаются. В пакете есть и другие методы коррекции.

https://xkcd.com/882/

![](https://imgs.xkcd.com/comics/significant.png)